In [3]:
%pylab
import numbers
import sys
sys.path.append(r'D:\opencv249\sources\modules\python\test')

import time
from datetime import datetime
from lxml import etree as et
from StringIO import StringIO
from primesense import openni2

import os
import cv2
import transformations as tf
'一些短别名:'
tf.e2m = tf.euler_matrix
tf.m2e = tf.euler_from_matrix
tf.q2m = tf.quaternion_matrix
tf.m2q = tf.quaternion_from_matrix
tf.e2q = tf.quaternion_from_euler
tf.q2e = tf.euler_from_quaternion
tf.m2r = tf.rotation_from_matrix
tf.r2m = tf.rotation_matrix
tf.a2q = tf.quaternion_about_axis
tf.qmul = tf.quaternion_multiply
tf.qinv = tf.quaternion_inverse

'angle <--> radian:'
a2r = np.pi / 180
r2a = 180 / np.pi

def q2a(qx, qy, qz, qw, vecLen = 4):
    '''
    参见 tf.quaternion_about_axis 实现;
    算法参见:
    http://www.euclideanspace.com/maths/geometry/rotations/conversions/angleToQuaternion/
    http://www.euclideanspace.com/maths/geometry/rotations/conversions/quaternionToAngle/
    
    测试代码：
    q=tf.a2q(0.123, (1,0,0))
    a=q2a(*q)
    print q
    print a
    '''
    if not ( isinstance(qx, numbers.Real) and isinstance(qy, numbers.Real) and isinstance(qz, numbers.Real) and isinstance(qw, numbers.Real) ):
        raise TypeError, 'Q:xyzw must be all numbers, not arrays / lists.'
        return
#     if not vecLen in (3, 4):
#         print 'vecLen 必须是 3 或者 4'
#         return
        
    angle = 2 * np.arccos(qw)
    'sin(angle/2):'
    sinA2 = np.sqrt(1 - qw * qw)
    x = qx / sinA2
    y = qy / sinA2
    z = qz / sinA2
    
    if vecLen == 4:
        return angle, x, y, z
    elif vecLen == 3:
        return x*angle, y*angle, z*angle
    else:
        raise ValueError, 'vecLen must be integer 3 or 4'
    pass


Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [17]:
'''
检查capg-imu【设备机身轴向】，方法：分别沿xyz三轴逆(+)顺(-)旋转, 观察欧拉角
'''
from datetime import datetime
from lxml import etree as et

'Y朝西。分别沿xyz三轴逆(+)顺(-)旋转, 观察欧拉角'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession-checkaxes.xml'
'Y朝北。其余同上'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession-checkaxes.ynorth.xml'

'Y朝北。其余同上。硬件ID：1191182427。居然plot结果是对的！与之前《姿态数据问题》报告结论相反！ //2016-1-3 04:00:34'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.20160103.test-duyu.xml'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.20160103.test-duyu.p.xml'

'Y朝北。其余同上, 重新操作一遍, 结果同上 √ //2016-1-3 10:24:13'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.chkAx.xml'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.chkAx.p.xml'

'Y朝北。其余同上，硬件ID: 1191182355。设备未校正，raw Rwxyz很奇怪, .p.xml看起来就好很多 //2016-1-3 10:23:36'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.chkAx.id1191182355.xml' 
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.chkAx.id1191182355.p.xml'

'Y朝北。其余同上，硬件ID: 1191182355。设备校正了！√ //2016-1-3 10:48:03'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.chkAx.id1191182355.calibed.xml'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.chkAx.id1191182355.calibed.p.xml'

'X朝北，先后单轴旋转+90°,-90°,归位。7.3°'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.xnorth+90.xml'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.xnorth+90.p.xml'

'1191182427. old X朝北, old-xyz ±90°, new-xyz ±90°. 验证信电系坐标系与杜宇&李启雷坐标系, 数据漂移.操! 146.14°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.xnorth.oxyz.nxyz.+90-90.xml'

'''
1191182427. old X朝西(略偏北), 一边卡在固定边界, old-xyz +360°. 
发现：
    1. 坐标系确实如前所想 (xyz)->(y, -x, z); 
    2. 此设备好像raw-ori存的就是以第一帧为参考系的？怪！确实：但又对不很齐，同见 xnorth.+360.xml
'''
'err: 12.68°'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.xwest.+360.xml'

'1191182427. old X朝北(略偏东), 一边卡在固定边界, old-xyz +360°, 16.25°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.xnorth.+360.xml'
'重新采, 运动如上, 起始状态差劲, 16.35°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.xnorth.+360.xml'
'又重新采, 这次保存.p.xml时候程序崩溃(没存成); 可能是没有 reset-session的原因。操！, 34.2°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.xnorth.+360.2.xml'
'静态, 1.1°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.xnorth.static.xml'
'单独标定, 重采静态, 6.5°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.xnorth.recalib.static.xml'


'''
1191182355, old X北偏东， 静止
发现：
    1. 起始不是单位阵, 偏的有点大, 见 xnorth.static.xml (X北略偏东)
    2. 关闭又打开, 换了个角度, 好点了, 见 xeast.static2.xml (X东略偏南)
    3. 仿照1重新采, 又好了, 原因不明, 见 xnorth.static3.xml
'''
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.xnorth.static.xml'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.xeast.static2.xml'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.xnorth.static3.xml'
'继续采。 2.4° //2016-1-12 22:18:42'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.static.xml'
'1.5°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.static.p.xml'
'继续采，开采之前使劲转了几下, 21.2° //2016-1-12 23:03:40'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.static.2.xml'
'0.523°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.static.2.p.xml'

'开采之前使劲转, 但是也reset-session. 0.9° //2016-1-12 23:06:05'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.static.3.xml'
'1.6°'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.static.3.p.xml'

'13.4°'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.xyz360.xml'
'0.9°'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.xyz360.p.xml'

'继续采，用程序 Release\app-poseGlobal.exe, xyz+360 //2016-1-14 16:41:42'
'11.647°'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.xyz360.20160114.xml'


'以为上次采的不错, 标定误差34°那个; 但是imu首尾误差居然有138.7°, 而对应cv首尾误差23° \
这是实际标定数据来吗? 忘了... //2016-1-12 20:59:23'
# capXmlPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103\CaptureSession.ir-cb-imu-precalib-20160103.xml'

'49.56°, 1912帧'
# capXmlPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160111\CaptureSession.ir-cb-imu-precalib-20160111.xml'

'232.447°, 2411帧，已经看不懂了。kinect磁场干扰？'
# capXmlPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160113\CaptureSession.ir-cb-imu-precalib-20160113.xml'

'''
+++++++++++++++2016-1-17 00:15:48
专门重新标定 1191182355, 怀疑之前标定有错误
发现杜宇归一化结果有问题, 见笔记：《zsens, motion-sensor, 动作捕捉系统, 杜宇》
采集程序： app-poseGlobal.exe - release
'''
'253帧。4.7°'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.ynorth.static.xml'
'1371帧。21.35°, 存在稳定漂移是不是设备或者标定方式有问题？'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.ynorth.static40s.xml'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.ynorth.xyz360.xml'

'id2427设备, 1539帧。25.9°'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.static40s.xml'

'id2355, 3183帧, 55.27°'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.ynorth.static100s.xml'

'''
+++++++++++++++2016-1-17 23:45:56
怀疑我的pc上代码有问题, 在移动工作站上采集数据, 再拷贝回来
发现：①没有漂移很好！②抖动很厉害（看起来，实际还凑合） ③pc上reset连击两下,就会快速漂移, 似乎最好: 关掉-reset-打开
推测：core.conf 配置文件的区别
'''
'0.76°'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.static100s.laptop.xml'
'0.42°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.xyz360.laptop.xml'

'在我的pc上改core.conf配置文件，重采数据(reconf, 且 app...poseGlobal), 1562帧, 6.83°, 说明有效果'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.ynorth.static60s.reconf.poseGlobal.xml'

'某reset-session导致快速漂移的数据, 255帧. 68.79°'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\resetSession.bad.xml'

'用app-poseGlobal.exe程序, 按照"关掉-reset-打开" 流程操作. 1837帧, 2.63°...说明poseGlobal初始化程序不是问题, 这个流程也是对的'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\resetSession.2427.poseGlobal.good.xml'

'用app.orig.exe程序, core.conf改回beta=0.1. 1912帧，3.03°...说明beta不是关键因素？'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\resetSession.2427.appOrig.good.xml'

'重新绑定固定，试图手眼标定。2941帧，id=1191182355，50.27°，//不能用, realtime 缺失'
capXmlPath = r'D:\Users\zhangxaochen\Documents\axxb-data\cb-imu-handeye-20160119\CaptureSession.cb-imu-handeye.xml'
'2498帧，id=1191182355，99.07°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\axxb-data\cb-imu-handeye-20160120\CaptureSession.cb-imu-handeye-20160120.xml'

'''+++++++++++++++2016-2-21 21:04:38，基本写完 loadXml-calibAcc-calcOri.cpp，
试图验证三种ori结果优劣：1. xml-自带Rwxyz; 2. 我自己调用filterUpdate; 3. 用 rgk-calib 的 new-Axyz; 4. nocalib; 5. 仅calib-gyro
'''
'/////////////////1. xml-自带Rwxyz：'
'2687帧, 1.3986°; 完全静止, 误差挺小'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.static.20160221.xml'
'2840帧, 2.47099°, 乱转, 最后回归原位. 结果不完美, 是不是挪动了?不确定'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.xyzshuffle.20160221.xml'
'1999帧, 0.776579°, 重新乱转, 非常小心注意开始/结束时刻位置姿态, 结果非常好'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.xyzshuffle2.20160221.xml'
'3926帧, 0.588748°, 接上次↑, 重新乱转, 归位. 但是忘了reset-session, 结果其实包含上次数据'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.xyzshuffle3.20160221.xml'
'/////////////////2. zcImpl们:'
'static, 1.36°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.static.20160221.zcImpl.xml'
'xyzshuffle, 2.35°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.xyzshuffle.20160221.zcImpl.xml'
'xyzshuffle2, 0.975°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.xyzshuffle2.20160221.zcImpl.xml'
'xyzshuffle3, 0.6286°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.xyzshuffle3.20160221.zcImpl.xml'

'/////////////////3. zcImpl且rgkCalib们:'
'static, 1.35°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.static.20160221.zcImpl.rgkCalib.xml'
'xyzshuffle, 2.3489°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.xyzshuffle.20160221.zcImpl.rgkCalib.xml'
'xyzshuffle2, 0.966°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.xyzshuffle2.20160221.zcImpl.rgkCalib.xml'
'xyzshuffle3, 0.6481858°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.xyzshuffle3.20160221.zcImpl.rgkCalib.xml'

'/////////////////4. 自己算, 且不做任何补偿/校正. nocalib //2016-3-1 14:39:39'
'static, 2.165°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.static.20160221.nocalib.xml'
'xyzshuffle, 2.552°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.xyzshuffle.20160221.nocalib.xml'
'xyzshuffle2, 1.966°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.xyzshuffle2.20160221.nocalib.xml'
'xyzshuffle3, 2.0513°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.xyzshuffle3.20160221.nocalib.xml'

'/////////////////5. 仅仅校正gyro呢？似乎不差啊！ //2016-3-1 10:53:05'
'static, 1.057°'
# capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.static.20160221.gyro.xml'
'xyzshuffle, 1.2698995°'
# capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.xyzshuffle.20160221.gyro.xml'
'xyzshuffle2, 1.156°'
# capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.xyzshuffle2.20160221.gyro.xml'
'xyzshuffle3, 0.96°'
# capXmlPath = r'D:\Users\zhangxaochen\Documents\Visual Studio 2010\Projects\rgkAccCalib\loadXml-calibAcc-calcOri\CaptureSession.2355.xyzshuffle3.20160221.gyro.xml'


print('capXmlPath:', capXmlPath)

psr = et.XMLParser(remove_blank_text = True)
tree = et.parse(capXmlPath, psr)
rt = tree.getroot()

node = rt[0][0]
print(node.attrib)
euls = []
agaxes = []
quas=[] #xyzw
for i, data in enumerate(node):
    attribs = data.attrib
    rxyzw = [float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz']), float(attribs['Rw'])]
    quas.append(rxyzw)
    
    eul = tf.q2e(rxyzw, axes='rxyz')
    euls.append(eul)
    
    agax = q2a(*rxyzw, vecLen=4)
    agaxes.append(agax)
    pass

quas = np.array(quas)
euls = np.array(euls)
agaxes = np.array(agaxes)

'计算误差: 0, -1帧差了多少度?'
# tf.q2m(quas[0]).T*tf.q2m(quas[-1]) #改为直接用四元数运算, 如下:
headIdx=30
tailIdx=-30
agax=q2a(*tf.qmul(tf.qinv(quas[headIdx]), quas[tailIdx]), vecLen=4)
print '首尾误差 0&-1 error:', min(agax[0], 2*np.pi-agax[0])* r2a
print 'det(tf.q2m(quas[0])):', det(tf.q2m(quas[0])) #虽然有些矩阵画出来三轴，看起来不像正交，但是确实没错

# close()
figure()
'四元数'
subplot(311)
plot(quas)
legend(list('xyzw'))
'欧拉角'
subplot(312)
plot(euls)
legend(list('xyz'))
'轴角表示xyz+angle'
subplot(313)
plot(agaxes[:, 1:]) #后三个量ax-xyz
plot(agaxes[:, 0]) #angle
legend(list('xyz')+['ang'])

'---------------肉眼观察某些关键帧的mat形式'
# inds=[316, 506, 645] #CaptureSession.chkAx.id1191182355.calibed.xml
inds=[11,] #CaptureSession.xnorth+90.xml

# for i in inds:
#     if i > len(node):
#         break
#     data=node[i]
#     attribs = data.attrib
#     rxyzw = [float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz']), float(attribs['Rw'])]
#     print(tf.q2m(rxyzw)[:3, :3])
#     pass

for i in inds:
    q = quas[i]
    print 'i:', i, tf.q2m(q)[:3, :3]
    pass
    
'---------------关键帧mat矩阵不好看，改为3D绘制坐标轴，动画演示'
from mpl_toolkits.mplot3d import axes3d
fig = figure()
ax = fig.add_subplot(111, projection='3d')

'添加xyz参考轴'
o3d=(0,0,0)
oxyz=np.eye(3)
ax.plot_wireframe(*zip(o3d, oxyz[0]), color='r')
ax.plot_wireframe(*zip(o3d, oxyz[1]), color='g')
ax.plot_wireframe(*zip(o3d, oxyz[2]), color='b')

wfx = wfy = wfz = None
# for i, dat in enumerate(node):
for i, q in enumerate(quas):
#     break
#     print i
    oldwfx, oldwfy, oldwfz = wfx, wfy, wfz
    
#     attribs = dat.attrib
#     rxyzw = [float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz']), float(attribs['Rw'])]
#     rmat = tf.q2m(rxyzw)[:3, :3]
    rmat = tf.q2m(q)[:3, :3]
    
    '调试看mat'
#     if i==108:
#         print i, rmat
#     elif i==136:
#         print i, rmat

#     print(rmat)
    vx = rmat[:,0]
    vy = rmat[:,1]
    vz = rmat[:,2]
    wfx = ax.plot_wireframe(*zip((0,0,0), vx), color='r')
    wfy = ax.plot_wireframe(*zip((0,0,0), vy), color='g')
    wfz = ax.plot_wireframe(*zip((0,0,0), vz), color='b')
    if oldwfx != None:
        ax.collections.remove(oldwfx)
        ax.collections.remove(oldwfy)
        ax.collections.remove(oldwfz)
        pass
    
#     fig.suptitle(str(i), fontsize=20)
#     tit = ax.set_title(str(i))
    ax.title.set_text('idx: %s'%i)
    plt.pause(0.03)
    pass



('capXmlPath:', 'D:\\Users\\zhangxaochen\\Documents\\Visual Studio 2010\\Projects\\rgkAccCalib\\loadXml-calibAcc-calcOri\\CaptureSession.2355.xyzshuffle3.20160221.nocalib.xml')
{'frames': '3926', 'phyId': '1191182355'}
首尾误差 0&-1 error: 2.05131327323
det(tf.q2m(quas[0])): 1.0
i: 11 [[-0.05621467  0.99691702 -0.05473906]
 [-0.99445157 -0.05102451  0.0919922 ]
 [ 0.08891556  0.05960666  0.99425403]]


KeyboardInterrupt: 

In [10]:
i=12
i

12

In [3]:
'非真“首尾”，手选特定帧做首尾'
agax=q2a(*tf.qmul(tf.qinv(quas[20]), quas[1202]), vecLen=4)
print '首尾误差 0&-1 error:', agax[0]* r2a

首尾误差 0&-1 error: 12.5269912904


In [21]:
'''
---------------使用手动挑选的帧进行手眼标定预处理！ 生成 axxb.m 的输入 xxx.cvAndImuExtr.csv
依赖: 
1. xxx.oni.txt 起始绝对时间戳;
2. xxx.frames2calib.txt 中存有手动从 oni 挑选的帧: (有效性, 帧序号, 相对时间戳)
3. 从以上俩txt得到绝对时间戳, 从CaptureSession.xxx.xml 中得到对应时间的IMU数据帧;
4. 整合记录 IR+IMU数据【16列】 xxx.ImuCorresp.csv (较原始): irFid irAbsTs imuA*3 imuM*3 imuG*3 imuR*4(wxyz) imuAbsTs
5. 重新整合 IR(R,t)(来自cam.xml)+IMU(R,t) 文件, 12列, 所以是: 
'''

import os, sys
import numpy as np
import time
from datetime import datetime
from lxml import etree as et
from StringIO import StringIO


'记录oni起始绝对时间戳的txt'
# oniBegtPath = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni.txt'
'记录从oni中手动挑选的帧: (有效性, 帧序号, 相对时间戳)'
# f2cPath = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni.frames2calib.txt'
'与oni对应的IMU(zsens采集程序) XML数据'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.ir-chessboard.xml'
'cv::cpp-example-calibration.exe标定的外参'
# extrFname = r'D:\Program Files (x86)\OpenNI\Samples\Bin\Release\CapturedFrames.ir-chessboard\cam.xml'

'内参配置文件, 唯一! 一个设备一个, 不该随标定数据而改变'
'17帧'
intrFname = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160113\CapturedFrames.ir-cb-imu-precalib-20160113\cam.xml'
print 'intrFname:', intrFname

'重构，组织到一个proj中去 2015-12-13 13:03:56'
projPath = r'D:\Users\zhangxaochen\Documents\ir-cb-imu-precalib'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160111'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160113'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\cb-imu-handeye-20160119'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\cb-imu-handeye-20160120'

'之前的固定方法，以及@杜宇坐标系约定下，理想的旋转矩阵'
# idealx=np.array([[0,1,0,0],[0,0,-1,0],[-1,0,0,0],[0,0,0,1]])
idealx=np.array([[0,1,0],[0,0,-1],[-1,0,0]])#,[0,0,0,1]])
'《手眼标定算法(AX=XB)测试报告》表6, 第一个ir-cb-imu-precalib.oni.frames2calib.cvAndImuExtr.csv'
idealx=np.array([[-0.2790, 0.8740, 0.3978],
                 [-0.1239, 0.3781, -0.9175],
                 [-0.9523, -0.3052, 0.0028],
])
t=np.eye(4)
t[:3,:3]=idealx
idealx=t
qx = tf.m2q(idealx)


os.chdir(projPath)
basePath, projName = os.path.split(projPath)
print 'projName:', projName
oniPath = projName+'.oni'
print 'oniPath:', oniPath
oniBegtPath = oniPath+'.txt'
print 'oniBegtPath:', oniBegtPath
f2cPath = oniPath+'.frames2calib.txt'
print 'f2cPath:', f2cPath
capXmlPath = 'CaptureSession.%s.xml'%projName
print 'capXmlPath:', capXmlPath
capFramesFolder = os.path.join(projPath, 'CapturedFrames.'+projName)
extrFname = os.path.join(capFramesFolder, 'cam.xml')
print 'extrFname:', extrFname

oniBegt = np.asscalar(np.loadtxt(oniBegtPath))
print 'oniBegt: %d'%oniBegt
f2cPathRoot, ext = os.path.splitext(f2cPath)

'从imu-xml中找到的对应帧记录，软同步，整合记录 IR+IMU数据 16列：(irFid, irAbsTs, imu-a3m3g3r4(wxyz)+ts)，以便之后处理'
'暂未用到'
f2cImuCorrespPath = f2cPathRoot + '.ImuCorresp.csv'
cvAndImuExtrPath = f2cPathRoot + '.cvAndImuExtr.csv'

f2calib = np.loadtxt(f2cPath, delimiter = '\t', skiprows = 1)

'解析CaptureSession.xml文件'
psr = et.XMLParser(remove_blank_text = True)
tree = et.parse(capXmlPath, psr)
rt = tree.getroot()
# print et.tostring(tree, pretty_print = True)
node = rt[0][0]
print 'node.attrib:', node.attrib
'总共data条目数量'
dataCnt = len(node)

# '在xml中找到时间戳对应的帧，保存3+3+3+4+realtime数据到 txt/csv'
# with open(f2cImuCorrespPath, 'w') as f2cIrImu: #改用 np.savetxt
'去掉了原f2c txt 第一列 "ok", 并改用绝对毫秒时间戳'
csvHeader = 'irFid\t irAbsTs\t imuA*3\t imuM*3\t imuG*3\t imuR*4(wxyz)\t imuAbsTs'
csvMat = [] #之后转为np-2d-array

'假设 frames2calib.txt 记录的帧序号从前往后是按时间序的, imuCurrPos 记录当前找到第几帧了'
imuCurrPos = 0
for r, row in enumerate(f2calib):
    #print row
    '要保存的新csv一行'
    csvRow = []
    isValid, irFid, irRelTs = row #irRelTs->相对时间戳(s)
    if isValid:
        #print irFid, irRelTs
        irAbsTs = oniBegt + irRelTs * 1e3 #类似 '1448203357080'
        irAbsDt = datetime.fromtimestamp(irAbsTs / 1e3)
        #print '%d'%irAbsTs
        
        csvRow += [irFid, irAbsTs]
        
        for i in range(imuCurrPos, dataCnt - 1):
            imuData = node[i]
            attribs = imuData.attrib
            imuDtStr = attribs['realtime']
            imuDtObj = datetime.strptime(imuDtStr, '%Y-%m-%d %H:%M:%S.%f')
            
            'imu-dt转为ts:'
            #imuAbsTs = time.mktime(imuDtObj.timetuple()) #丢失毫秒
            imuAbsTs = (imuDtObj - datetime(1970,1,1)).total_seconds() * 1e3
            

            '第 i+1 条目的时间戳'
            imuDtObj2 = datetime.strptime(node[i+1].attrib['realtime'], '%Y-%m-%d %H:%M:%S.%f')

            if imuDtObj <= irAbsDt <= imuDtObj2:
                #print r, row, i
                #print imuDtObj, irAbsDt, imuDtObj2

                csvRow += [float(attribs['Ax']), float(attribs['Ay']), float(attribs['Az'])
                          ,float(attribs['Mx']), float(attribs['My']), float(attribs['Mz'])
                          ,float(attribs['Gx']), float(attribs['Gy']), float(attribs['Gz'])
                          ,float(attribs['Rw']), float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz'])
                          #,imuDtStr # < irDt
                          ,imuAbsTs #改用ts-float, 虽然肉眼难读懂
                          ]
                
                imuCurrPos = i
                break
                pass
            pass
        
        csvMat.append(csvRow)
        pass
    pass

csvMat = np.asarray(csvMat) #没有也行？！
# print csvMat

'保存到 xxx.ImuCorresp.csv文件, 不用 with-open'
np.savetxt(f2cImuCorrespPath, csvMat, fmt='%f', header=csvHeader)

'---------------5. 重新整合 IR(R,t)(来自cam.xml)+IMU(R,t) 文件, 12列'
psr = et.XMLParser(remove_blank_text = True, remove_comments=True)
tree = et.parse(extrFname, psr)
rt = tree.getroot()
cameraMatrixNode = rt[8]
# print cameraMatrixNode
distortionCoefficientsNode = rt[9]
cvExtrParamsNode = rt[12]
cvExtrParamsDataNode = cvExtrParamsNode[3]
# print extrParams.text
cvExtrParams = np.genfromtxt(StringIO(cvExtrParamsDataNode.text)) #shape=(N,2)
'IR--OpencvCalib 弄成6列, 6-tuples (rotation vector + translation vector) '
cvExtrParams = cvExtrParams.reshape(-1, 6)

'''
---------------策略①：rot-vec为轴角表示
cvExtrParams 本已经是轴角表示
需要：imu-xml的Rwxyz -> AxisAngle; t-位移fake为 (0,0,0)
可以！matlab中用 light field toolbox:: rodrigues 预处理. 详见 axxb.m
'''
imuExtrp = np.array([q2a(row[12], row[13], row[14], row[11], vecLen=3) for row in csvMat])
# print imuExtrp
# print np.hstack((imuExtrp, np.zeros_like(imuExtrp)))
imuExtrp = np.hstack([imuExtrp, np.zeros_like(imuExtrp)] )
# print imuExtrp
cvAndImuExtr = np.hstack([cvExtrParams, imuExtrp])
# print cvAndImuExtr
np.savetxt(cvAndImuExtrPath, cvAndImuExtr, fmt='%f')


'---------------策略②：rot-vec为四元数表示'
#do-sth.


intrFname: D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160113\CapturedFrames.ir-cb-imu-precalib-20160113\cam.xml
projName: cb-imu-handeye-20160120
oniPath: cb-imu-handeye-20160120.oni
oniBegtPath: cb-imu-handeye-20160120.oni.txt
f2cPath: cb-imu-handeye-20160120.oni.frames2calib.txt
capXmlPath: CaptureSession.cb-imu-handeye-20160120.xml
extrFname: D:\Users\zhangxaochen\Documents\axxb-data\cb-imu-handeye-20160120\CapturedFrames.cb-imu-handeye-20160120\cam.xml
oniBegt: 1453223038968
node.attrib: {'frames': '2498', 'phyId': '1191182355'}


'---------------\xe7\xad\x96\xe7\x95\xa5\xe2\x91\xa1\xef\xbc\x9arot-vec\xe4\xb8\xba\xe5\x9b\x9b\xe5\x85\x83\xe6\x95\xb0\xe8\xa1\xa8\xe7\xa4\xba'

In [2]:
'''
//2016-1-15 15:21:33
仅对手眼标定的关键帧绘制3D坐标轴, 观察IMU+camera 两坐标轴联动状态
'''
import os
from mpl_toolkits.mplot3d import axes3d

'+++++++++++++++imu->cv转换矩阵, capg-imu硬件, 且在我目前固定方式下: //2016-1-15 19:36:07'
cvAndImuExtrPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160113\ir-cb-imu-precalib-20160113.oni.frames2calib.cvAndImuExtr.csv'
# 'idealx'
# yy=np.array([[0,1,0],
#              [0,0,-1],
#              [-1,0,0]])#,[0,0,0,1]])

# yy=np.array([[-0.2507,    0.9680,   -0.0100],
#              [0.2601,    0.0574,   -0.9639],
#              [-0.9325,   -0.2443,   -0.2662]])

cvAndImuExtrPath = r'D:\Users\zhangxaochen\Documents\axxb-data\cb-imu-handeye-20160120\cb-imu-handeye-20160120.oni.frames2calib.cvAndImuExtr.csv'

# '误用某交叉验证的标定结果, 标定子集小, 重来'
# # yy=np.array([[-0.1131, 0.9935,-0.0142],
# #              [0.2748, 0.0176,-0.9614],
# #              [-0.9548, -0.1126,-0.2750]])

# '用此数据全集 11张图'
# yy=np.array([[-0.0206, 0.9998, 0.0021],
#              [0.1616, 0.0054,-0.9868],
#              [-0.9866,-0.0200,-0.1617]])

'+++++++++++++++@王镇 //作对比, 理论上应该不错 //注意@王镇 & @Android 都使用 SWD地球坐标系 //2016-1-15 19:37:06'
# cvAndImuExtrPath = r'D:\Users\zhangxaochen\Documents\axxb-data\Domke\NVan\cvAndImuExtr.wz.csv'
# 'idealx'
# yy=np.array([[0,-1,0],
#              [-1,0,0],
#              [0,0,-1]])
# '《手眼标定算法(AX=XB)测试报告》表3-序号5'
# yy=np.array([[0.0181, -0.9998, -0.0127],
#              [-0.9998, -0.0180, -0.0031],
#              [0.0029,  0.0128, -0.9999]])

'---------------改用自动读取 axxb.m 输出的 handeyeResult.txt'
fpath, fname = os.path.split(cvAndImuExtrPath)
handeyeResFname = os.path.join(fpath, 'handeyeResult.txt')
assert os.path.isfile(handeyeResFname), 'handeyeResFname.txt must exists in the data-project-folder'
print 'handeyeResFname:', handeyeResFname
yy = np.loadtxt(handeyeResFname, delimiter=',')
print 'yy:', yy
t = np.eye(4)
t[:3,:3] = yy
qx = tf.m2q(t)
print 'qx:', qx

'读文件数据:'
cvAndImuExtr = np.loadtxt(cvAndImuExtrPath)

'定位到csv目录, 后面存png要存到对应目录'
basePath, fname = os.path.split(cvAndImuExtrPath)
print 'basePath:', basePath
print 'fname:', fname

os.chdir(basePath)

fig = figure()
ax = fig.add_subplot(111, projection='3d')

'cv作为基准原点，添加xyz参考轴'
o3d=(0,0,0)
pimu3d=(0,2,0) #IMU基准点
oxyz=np.eye(3)
for i, cl in zip(oxyz, list('rgb')):
    ax.plot_wireframe(*zip(o3d, i), color=cl, linewidth=2)
    pass

'需要删除的旧线条句柄数组'
oldlines=[]
for i, row in enumerate(cvAndImuExtr):
    irAgax = row[0:3]
    imuAgax = row[6:9]
    print 'i:', i, irAgax
    '相机坐标系转换: w->b / cb->camera; δ=R0*Ri.T'
    irRmat = cv2.Rodrigues(irAgax)[0]
    'IMU坐标系转换: b->w / imu->ENU; δ=R0.T*Ri'
    imuRmat = cv2.Rodrigues(imuAgax)[0]
    
    if i==0:
        irR0 = irRmat
        imuR0 = imuRmat
        pass
    '都转到相对i0坐标系下：'
    irDrmat = irR0.dot(irRmat.T)
    imuDrmat = imuR0.T.dot(imuRmat)
    'imu还要继续转到cv坐标系下：'
    imuDrmat = yy.dot(imuDrmat).dot(yy.T)
    
    irvx = irDrmat[:,0]
    irvy = irDrmat[:,1]
    irvz = irDrmat[:,2]
    
    imuvx = imuDrmat[:,0]
    imuvy = imuDrmat[:,1]
    imuvz = imuDrmat[:,2]
    
    '+++++++++++++++这里开始绘制更新'
    if oldlines:
        for ln in oldlines:
            ax.collections.remove(ln)
        oldlines=[]
        pass
    
    'plot cv-ir-camera:'
    #o3d = (0, i*2, 0)
    oldlines.append(ax.plot_wireframe(*zip(o3d, o3d+irvx), color='r'))
    oldlines.append(ax.plot_wireframe(*zip(o3d, o3d+irvy), color='g'))
    oldlines.append(ax.plot_wireframe(*zip(o3d, o3d+irvz), color='b'))
    
    'plot imu:'
    #pimu3d = (2,i*2,0) #IMU基准点
    oldlines.append(ax.plot_wireframe(*zip(pimu3d, pimu3d+imuvx), color='r'))
    oldlines.append(ax.plot_wireframe(*zip(pimu3d, pimu3d+imuvy), color='g'))
    oldlines.append(ax.plot_wireframe(*zip(pimu3d, pimu3d+imuvz), color='b'))
    
    ax.title.set_text('idx: %s'%i)
    plt.pause(.1)
    plt.savefig('cv-imu-%d.png'%i, bbox_inches='tight')

#     draw()
#     raw_input('') #没法用作键盘响应, 加了draw()也不行
    pass

handeyeResFname: D:\Users\zhangxaochen\Documents\axxb-data\cb-imu-handeye-20160120\handeyeResult.txt
yy: [[-0.02063915  0.99978475  0.00211752]
 [ 0.16160872  0.00542629 -0.98684   ]
 [-0.98663907 -0.02002533 -0.16168593]]
qx: [ 0.53282748  0.54492003 -0.46193261  0.45362463]
basePath: D:\Users\zhangxaochen\Documents\axxb-data\cb-imu-handeye-20160120
fname: cb-imu-handeye-20160120.oni.frames2calib.cvAndImuExtr.csv
i:

d:\Anaconda2x86\lib\site-packages\matplotlib\collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
d:\Anaconda2x86\lib\site-packages\matplotlib\backend_bases.py:2399: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


 0 [ 0.191859  0.182141 -3.110943]
i: 1 [-0.556549 -0.679217  2.826994]
i: 2 [-0.483481  0.187481  2.994873]
i: 3 [ 0.639028 -0.880443 -2.776253]
i: 4 [-0.434159 -0.516037 -2.804019]
i: 5 [-0.598897  0.418566 -2.881637]
i: 6 [-0.199901 -0.758827  3.016534]
i: 7 [ 0.123927 -0.646111  2.55541 ]
i: 8 [ 0.823434  0.817712 -2.456024]
i: 9 [ 0.743073 -0.58572  -2.880611]
i: 10 [ 0.100666  0.067824 -3.134819]


In [24]:
'''
---------------仅仅 read_frame, 不读实际buffer, 之后会不会保持内存, 为的是尽力避免跳帧 //2016-1-10 23:54:08
'''

from primesense import openni2

openni2.initialize()     # can also accept the path of the OpenNI redistribution
# oniFname = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni'
# oniFname = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103\ir-cb-imu-precalib-20160103.oni' #此文件 dep 全黑
'用上面↑cell的oniPath'
dev = openni2.Device.open_file(oniPath)

print 'dev.get_device_info:', dev.get_device_info()
print 'dev.get_sensor_info:', dev.get_sensor_info(1)

depth_stream = dev.create_depth_stream()
depth_stream.start()
ir_stream = dev.create_ir_stream()
ir_stream.start()

dfs = []
irfs = []

depFcnt = depth_stream.get_number_of_frames()
print 'depFcnt:', depFcnt

'导致 dfs, irfs 必然长度为 depFcnt, 但可能存在尾部循环:'
for i in range(depFcnt):
    print i,
    if i > 1000:
        break
        
    depframe = depth_stream.read_frame()
    depfid = depframe.frameIndex
    depRelTs = depframe.timestamp #微秒，非毫秒
    dfs.append(depframe)
    #print allclose(np.asarray(depframe.get_buffer_as_uint16()), 0) #对于 ir-cb-imu-precalib-20160103.oni, 得到 True
    
    irframe = ir_stream.read_frame()
    irfid = irframe.frameIndex
    irRelTs = irframe.timestamp
    irfs.append(irframe)
    
    pass

'这里不能释放, 否则下面的cell就不能用了'
# depth_stream.stop()
# openni2.unload()

dev.get_device_info: OniDeviceInfo(uri = 'cb-imu-handeye-20160120.oni', vendor = 'PrimeSense, Ltd.', name = 'oni File', usbVendorId = 0, usbProductId = 0)
dev.get_sensor_info: <primesense.openni2.SensorInfo object at 0x08B22190>
depFcnt: 2137
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216

'\xe8\xbf\x99\xe9\x87\x8c\xe4\xb8\x8d\xe8\x83\xbd\xe9\x87\x8a\xe6\x94\xbe, \xe5\x90\xa6\xe5\x88\x99\xe4\xb8\x8b\xe9\x9d\xa2\xe7\x9a\x84cell\xe5\xb0\xb1\xe4\xb8\x8d\xe8\x83\xbd\xe7\x94\xa8\xe4\xba\x86'

In [6]:
'''
处理上个cell得到的 dep/ir-frame, 读实际buffer //2016-1-10 23:54:57
用 dep-frame
+++++++++++++++测试，播放视频演示
'''

im = None
for i, df in enumerate(dfs):
    if i > 100:
        break
    print 'i:', i
    
    depdata = df.get_buffer_as_uint16() #c_ushort_Array_307200
    depdata = np.asarray(depdata) #1D, dtype=uint16
    depdata = depdata.reshape(df.height, df.width)
    
    #imshow(depdata) #叠加绘制导致越来越慢, 弃用!
    if im is None:
        im = imshow(depdata)
    else:
        im.set_data(depdata)
    
    pause(.03)
    draw()

i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
i: 20
i: 21
i: 22
i: 23
i: 24
i: 25
i: 26
i: 27
i: 28


KeyboardInterrupt: 

In [6]:
'''
处理上个cell得到的 dep/ir-frame, 读实际buffer //2016-1-10 23:54:57
用 ir-frame
用cv2在这里外定标
+++++++++++++++测试 N图像 calibrateCamera, 一起标定, 很慢, 后面采用 solvePnP
'''

square_size = 1.
pattern_size = (5,8)
pattern_points = np.zeros( (np.prod(pattern_size), 3), np.float32 )
pattern_points[:,:2] = np.indices(pattern_size).T.reshape(-1, 2)
pattern_points *= square_size

obj_points = []
img_points = []


im = None
for i, irf in enumerate(irfs):
    if i > 20:
        break
    print 'i:', i,
    
    irdata = irf.get_buffer_as_uint16() #ir也是16, c_ushort_Array_307200; 8不行
    irdata = np.asarray(irdata) #1D, dtype=uint16
    irdata = irdata.reshape(irf.height, irf.width)
    '转为uint8灰度图:'
    irdata = irdata*255./np.max(irdata)
    irdata = irdata.astype(uint8)

    found, corners = cv2.findChessboardCorners(irdata, pattern_size)
    viz = cv2.cvtColor(irdata, cv2.COLOR_GRAY2BGR)
    cv2.drawChessboardCorners(viz, pattern_size, corners, found)
    
    if not found:
        print 'chessboard not found'
        continue
    img_points.append(corners.reshape(-1, 2))
    obj_points.append(pattern_points)

#     #imshow(depdata) #叠加绘制导致越来越慢, 弃用!
#     if im is None:
#         im = imshow(viz)#, cmap='gray')
#     else:
#         im.set_data(viz)
    
#     #pause(.03)
#     draw()
    cv2.imshow('cv2-calib', viz)
    cv2.waitKey(0)
    pass

cv2.destroyAllWindows()
print ''

'N图像一起标定, N太大会卡死(<30):'
rms, camera_matrix, dist_coefs, rvecs, tvecs = \
cv2.calibrateCamera(obj_points, img_points, (irf.width, irf.height))
print 'rms:', rms, camera_matrix

i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 i: 10 i: 11 i: 12 i: 13 i: 14 i: 15 i: 16 i: 17 i: 18 i: 19 i: 20 
rms: 0.222749971846 [[ 554.46140949    0.          314.76433778]
 [   0.          547.80000916  255.95597547]
 [   0.            0.            1.        ]]


In [43]:
'''
接上个cell, 但是改用 solvePnP //2016-1-11 03:03:03
用 ir-frame
用cv2在这里外定标
采用 solvePnP, 而不用 calibrateCamera。
输出每一帧cv-extrinsic结果，绘制dQuas(相对第0帧)
+++++++++++++++此cell必要, 输出 dQuas
'''

from lxml import etree as et

square_size = 1.
pattern_size = (5,8)
pattern_points = np.zeros( (np.prod(pattern_size), 3), np.float32 )
pattern_points[:,:2] = np.indices(pattern_size).T.reshape(-1, 2)
pattern_points *= square_size

# obj_points = []
# img_points = []

'+++++++++++++++预先从某 cam.xml 读入内参+畸变'
'按理说 cam.xml 保留一个最准的就行 //+++++++++++++++不用 extrFname这个变量 //2016-1-21 10:54:11'
# extrFname = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103\CapturedFrames.ir-cb-imu-precalib-20160103\cam.xml'
print 'intrFname:', intrFname
psr = et.XMLParser(remove_blank_text = True, remove_comments=True)
tree = et.parse(intrFname, psr)
rt = tree.getroot()
cameraMatrixNode = rt[8]
# print cameraMatrixNode
distortionCoefficientsNode = rt[9]
camMatDataNode = cameraMatrixNode[3]
distDataNode = distortionCoefficientsNode[3]
intrMat = np.genfromtxt(StringIO(camMatDataNode.text.replace('\n', ''))).reshape(3,3)
distoMat = np.genfromtxt(StringIO(distDataNode.text.replace('\n', '')))

'+++++++++++++++求解相机变换矩阵solvePnP'
rmat0 = None
dQuas = [] #delta-qua = R0*Ri.T

im = None
for i, irf in enumerate(irfs):
    '测试调试用 i:'
#     if i > 130:
#         break
    print 'i:', i,
    
    irdata = irf.get_buffer_as_uint16() #ir也是16, c_ushort_Array_307200; 8不行
    irdata = np.asarray(irdata) #1D, dtype=uint16
    irdata = irdata.reshape(irf.height, irf.width)
    '转为uint8灰度图:'
    irdata = irdata*255./np.max(irdata)
    irdata = irdata.astype(uint8)

    found, corners = cv2.findChessboardCorners(irdata, pattern_size)
    viz = cv2.cvtColor(irdata, cv2.COLOR_GRAY2BGR)
    cv2.drawChessboardCorners(viz, pattern_size, corners, found)
    
    '务必存档, 离线观察'
    cv2.imwrite('cv-cb-corner-%s.png'%i, viz)
    
    if not found:
        #print 'chessboard not found'
        print '×',
        '找不到角点时候，用前面好的帧填充：'
        if Qi is not None:
            dQuas.append(Qi)
            pass
        cv2.imshow('cv2-calib', viz)
        key = cv2.waitKey(01)
        if key==27:
            break
        continue
        
    corners = corners.reshape(-1, 2)
    #     img_points.append(corners)
    #     obj_points.append(pattern_points)
    ok, rvec, tvec = cv2.solvePnP(pattern_points, corners, intrMat, distoMat)
    #print 'ok:', ok, rvec, tvec #3*1， 3*1
    
    if rmat0 is None:
        rmat0 = cv2.Rodrigues(rvec)[0]
    
    rmati = cv2.Rodrigues(rvec)[0]
    'dR = R0*Ri.T'
    dRmat = rmat0.dot(rmati.T)
    '3x3 -> 4x4'
    t = np.eye(4)
    t[:3,:3] = dRmat
    Qi = tf.m2q(t)
    dQuas.append(Qi)

#     #imshow(depdata) #叠加绘制导致越来越慢, 弃用!
#     if im is None:
#         im = imshow(viz)#, cmap='gray')
#     else:
#         im.set_data(viz)
    
#     #pause(.03)
#     draw()
    cv2.imshow('cv2-calib', viz)
    key = cv2.waitKey(01)
    if key==27: #Esc, √
        break
    pass

cv2.destroyAllWindows()

dQuas = np.asarray(dQuas)

'N个图像一起标定, N太大会卡死(<30):'
# rms, camera_matrix, dist_coefs, rvecs, tvecs = \
# cv2.calibrateCamera(obj_points, img_points, (irf.width, irf.height))
# print 'rms:', rms, camera_matrix

'标定数据的运动, 刻意首尾姿态尽量对齐, 查看误差'
agax=q2a(*tf.qmul(tf.qinv(dQuas[0]), dQuas[-1]), vecLen=4)
print '首尾误差 0&-1 error:', agax[0]* r2a


 i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 i: 10 i: 11 i: 12 i: 13 i: 14 i: 15 i: 16 i: 17 i: 18 i: 19 i: 20 i: 21 i: 22 i: 23 i: 24 i: 25 i: 26 i: 27 i: 28 i: 29 i: 30 i: 31 i: 32 i: 33 i: 34 i: 35 i: 36 i: 37 i: 38 i: 39 i: 40 i: 41 i: 42 i: 43 i: 44 i: 45 i: 46 i: 47 i: 48 i: 49 i: 50 i: 51 i: 52 i: 53 i: 54 i: 55 i: 56 i: 57 i: 58 i: 59 i: 60 i: 61 i: 62 i: 63 i: 64 i: 65 i: 66 i: 67 i: 68 i: 69 i: 70 i: 71 i: 72 i: 73 i: 74 i: 75 i: 76 i: 77 i: 78 i: 79 i: 80 i: 81 i: 82 i: 83 i: 84 i: 85 i: 86 i: 87 i: 88 i: 89 i: 90 i: 91 i: 92 i: 93 i: 94 i: 95 i: 96 i: 97 i: 98 i: 99 i: 100 i: 101 i: 102 i: 103 i: 104 i: 105 i: 106 i: 107 i: 108 i: 109 i: 110 i: 111 i: 112 i: 113 i: 114 i: 115 i: 116 i: 117 i: 118 i: 119 i: 120 i: 121 i: 122 i: 123 i: 124 i: 125 i: 126 i: 127 i: 128 i: 129 i: 130 i: 131 i: 132 i: 133 i: 134 i: 135 i: 136 i: 137 i: 138 i: 139 i: 140 i: 141 i: 142 i: 143 i: 144 i: 145 i: 146 i: 147 i: 148 i: 149 i: 150 i: 151 i: 152 i: 153 × i: 154 × i: 155 × i: 156 × i:

In [38]:
extrFname

'D:\\Users\\zhangxaochen\\Documents\\axxb-data\\cb-imu-handeye-20160120\\CapturedFrames.cb-imu-handeye-20160120\\cam.xml'

In [67]:
'''
暂不管上面IR-IMU-calib过程，直接用 IMU 数据做初值
读取oni, depth(not IR)为主轴, 提取imu-xml对应时间的数据帧
用到 https://pypi.python.org/pypi/primesense
+++++++++++++++生成 dQuasImu 矩阵, 并输出到 syntheticRT.txt 文件
'''
import numpy as np
from datetime import datetime
from lxml import etree as et
from primesense import openni2

'与oni对应的IMU(zsens采集程序) XML数据'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.ir-chessboard.xml'
# capXmlPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103\CaptureSession.ir-cb-imu-precalib-20160103.xml'
'解析CaptureSession.xml文件'
psr = et.XMLParser(remove_blank_text = True)
tree = et.parse(capXmlPath, psr)
rt = tree.getroot()
# print et.tostring(tree, pretty_print = True)
node = rt[0][0]
print node.attrib
'总共data条目数量'
dataCnt = len(node)

'记录oni起始绝对时间戳的txt'
# oniBegtPath = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni.txt'
# oniBegtPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103\ir-cb-imu-precalib-20160103.oni.txt'
oniBegt = np.asscalar(np.loadtxt(oniBegtPath))
print 'oniBegt: %d'%oniBegt

# openni2.initialize()     # can also accept the path of the OpenNI redistribution

# oniFname = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni'
# dev = openni2.Device.open_file(oniFname)
# # dev = openni2.Device.open_any()
# print 'dev.get_device_info:', dev.get_device_info()
# print 'dev.get_sensor_info:', dev.get_sensor_info(1)

# depth_stream = dev.create_depth_stream()
# depth_stream.start()
# ir_stream = dev.create_ir_stream()
# ir_stream.start()

rmat0imu = q0imu = None
dQuasImu = []
"存储 XB'X'之后的q, 用作与cv结果对比"
newDquasImu = []

'初始IMU三轴与cv坐标轴的偏差旋转：'
dQuasImuInit = np.eye(3)

'生成以前协议的 syntheticRT.txt 文件, 共12列: tvec*3, rmat*9; 对应pcl参数 -csv_rt_hint'
imuRt12 = [] #之后转为np-2d-array
'imuCurrPos 记录当前在imu-xml找到第几帧了'
imuCurrPos = 0
depFcnt = depth_stream.get_number_of_frames()
# for i in range(depFcnt):
#     depframe = depth_stream.read_frame()
#     depfid = depframe.frameIndex
#     depRelTs = depframe.timestamp #微秒，非毫秒

#     irframe = ir_stream.read_frame()
#     irfid = irframe.frameIndex
#     irRelTs = irframe.timestamp

'用之前预读得到的：'
for i, irf in enumerate(irfs):
    dataframe = irf #这次用 ir-frame 做主数据流
    fid = dataframe.frameIndex
    relTs = dataframe.timestamp

    print '--:', i, fid

    '获取真实数据帧'
    frame_data = dataframe.get_buffer_as_uint16()
    #dat=np.frombuffer(frame_data, dtype=np.uint16)
    dat=np.array(frame_data) #跟上面等价？
    #print fid, relTs, dat[:20]
    
    absTs = oniBegt + relTs/1e3 #毫秒
    absDt = datetime.fromtimestamp(absTs / 1e3)
    #print absDt
    
    'tvec*3, rmat*9 (共12列); tvec 目前fake: 0,0,0'
    imuRtRow  = [0]*3
    
    '此处故意用下标i, 虽然不好:'
    for j in range(imuCurrPos, dataCnt - 1):
        imuData = node[j]
        attribs = imuData.attrib
        imuDtStr = attribs['realtime']
        imuDtObj = datetime.strptime(imuDtStr, '%Y-%m-%d %H:%M:%S.%f')

        'imu-dt转为ts:'
        #imuAbsTs = time.mktime(imuDtObj.timetuple()) #丢失毫秒
        imuAbsTs = (imuDtObj - datetime(1970,1,1)).total_seconds() * 1e3

        '第 j+1 条目的时间戳'
        imuDtObj2 = datetime.strptime(node[j+1].attrib['realtime'], '%Y-%m-%d %H:%M:%S.%f')

        if imuDtObj <= absDt <= imuDtObj2:
            print '==:', j
            rxyzw = [float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz']), float(attribs['Rw'])]
            '此矩阵未标定, 坐标系可能与cv坐标系定义不同'
            rmat = tf.q2m(rxyzw)[:3, :3] #本来得到的是 4x4 矩阵
            
            '起始帧:'
            if rmat0imu is None:
                q0imu = rxyzw
                rmat0imu = rmat
                print 'q0imu:', q0imu

            'i转到0: B0.T*Bi'
            dQiImu = tf.qmul(tf.qinv(q0imu), rxyzw)
            dQuasImu.append(dQiImu)

            "存储 XB'X'之后的q, 用作与cv结果对比"
            newq = tf.qmul(qx, tf.qmul(dQiImu, tf.qinv(qx)))
            newDquasImu.append(newq)

            print 'dQiImu:', dQiImu
            
            imuRtRow.extend(rmat.flatten())
            
            imuCurrPos = j
            imuRt12.append(imuRtRow)
            
            pass
        pass
    
    #imuRt12.append(imuRtRow) #移到内循环 innerFor->if
    pass

dQuasImu = np.asarray(dQuasImu)
newDquasImu = np.asarray(newDquasImu)

# depth_stream.stop()
# ir_stream.stop()
# openni2.unload()

# ---------------
print np.array(imuRt12).shape #==>(N, 12)

'保存到 txt 文件, 需要手动拷贝到oni导出为png序列的目录'
np.savetxt('syntheticRT.txt', imuRt12, fmt='%.12f')

{'frames': '2498', 'phyId': '1191182355'}
oniBegt: 1453223038968
--: 0 1
==: 300
q0imu: [-0.107486613095, -0.105457261205, 0.171980515122, 0.973523676395]
dQiImu: [  1.38777878e-17  -1.38777878e-17   0.00000000e+00   1.00000000e+00]
--: 1 2
==: 305
dQiImu: [-0.00153405  0.01122851 -0.00245381  0.99993265]
--: 2 3
==: 305
dQiImu: [-0.00153405  0.01122851 -0.00245381  0.99993265]
--: 3 4
==: 306
dQiImu: [-0.00487585  0.01556502 -0.00267956  0.99986331]
--: 4 5
==: 308
dQiImu: [ 0.0059488   0.00963108 -0.00562643  0.9999201 ]
--: 5 6
==: 309
dQiImu: [ 0.00684329  0.00500852 -0.00633224  0.99994392]
--: 6 7
==: 309
dQiImu: [ 0.00684329  0.00500852 -0.00633224  0.99994392]
--: 7 8
==: 310
dQiImu: [ 0.0051725   0.00338077 -0.0076523   0.99995164]
--: 8 9
==: 312
dQiImu: [ 0.00478149  0.00224561 -0.00855209  0.99994946]
--: 9 10
==: 312
dQiImu: [ 0.00478149  0.00224561 -0.00855209  0.99994946]
--: 10 11
==: 313
dQiImu: [ 0.00178374 -0.002836   -0.00866142  0.99995682]
--: 11 12
==: 315
dQiImu

In [68]:
'分别画两张图'
plot(dQuas)
legend(list('xyzw'))
figure()
plot(newDquasImu)
legend(list('xyzwabcd'))

'两个画在一张图'
figure()
plot(dQuas)
plot(newDquasImu)
legend(list('xyzwabcd'))


In [76]:
dQuas.shape, newDquasImu.shape

((1001, 4), (1001, 4))

In [98]:
'''
---------------3D绘制所有帧坐标轴，与上面绘制四元数进行对照。//2016-1-22 09:41:08

'''
fig = figure()
ax = fig.add_subplot(111, projection='3d')

'cv作为基准原点，添加xyz参考轴'
o3d=(0,0,0)
pimu3d=(0,2,0) #IMU基准点
oxyz=np.eye(3)
for i, cl in zip(oxyz, list('rgb')):
    ax.plot_wireframe(*zip(o3d, i), color=cl, linewidth=2)
    pass

'需要删除的旧线条句柄数组'
oldlines=[]
for i, (qcv, qimu) in enumerate(zip(dQuas, newDquasImu)):#[582:583]:
#     irAgax = row[0:3]
#     imuAgax = row[6:9]
#     print 'i:', i, irAgax
#     '相机坐标系转换: w->b / cb->camera; δ=R0*Ri.T'
#     irRmat = cv2.Rodrigues(irAgax)[0]
#     'IMU坐标系转换: b->w / imu->ENU; δ=R0.T*Ri'
#     imuRmat = cv2.Rodrigues(imuAgax)[0]
    irRmat = tf.q2m(qcv)[:3, :3]
    imuRmat = tf.q2m(qimu)[:3, :3]
    
#     if i==0:
#         irR0 = irRmat
#         imuR0 = imuRmat
#         pass
#     '都转到相对i0坐标系下：'
#     irDrmat = irR0.dot(irRmat.T)
#     imuDrmat = imuR0.T.dot(imuRmat)
#     'imu还要继续转到cv坐标系下：'
#     imuDrmat = yy.dot(imuDrmat).dot(yy.T)
    irDrmat = irRmat
    imuDrmat = imuRmat
    
    irvx = irDrmat[:,0]
    irvy = irDrmat[:,1]
    irvz = irDrmat[:,2]
    
    imuvx = imuDrmat[:,0]
    imuvy = imuDrmat[:,1]
    imuvz = imuDrmat[:,2]
    
    '+++++++++++++++这里开始绘制更新'
    if oldlines:
        for ln in oldlines:
            ax.collections.remove(ln)
        oldlines=[]
        pass
    
    'plot cv-ir-camera:'
    #o3d = (0, i*2, 0)
    oldlines.append(ax.plot_wireframe(*zip(o3d, o3d+irvx), color='r'))
    oldlines.append(ax.plot_wireframe(*zip(o3d, o3d+irvy), color='g'))
    oldlines.append(ax.plot_wireframe(*zip(o3d, o3d+irvz), color='b'))
    
    'plot imu:'
    #pimu3d = (2,i*2,0) #IMU基准点
    oldlines.append(ax.plot_wireframe(*zip(pimu3d, pimu3d+imuvx), color='r'))
    oldlines.append(ax.plot_wireframe(*zip(pimu3d, pimu3d+imuvy), color='g'))
    oldlines.append(ax.plot_wireframe(*zip(pimu3d, pimu3d+imuvz), color='b'))
    
    ax.title.set_text('idx: %s'%i)
    plt.pause(.01)
    plt.savefig('cv-imu-%d.png'%i, bbox_inches='tight')

#     draw()
#     raw_input('') #没法用作键盘响应, 加了draw()也不行
    pass

In [74]:
'''
+++++++++++++++接耿老师要求, 必须用@任光阔@王镇的加速度校正方法 //2016-1-22 15:44:18
计划采用一组14段数据进行校正(6轴+8半球)
旧的采集格式(zsens)转码到新数据格式(Android-zhangxaochen之前的程序)
'''

from lxml import etree as et
import sys
sys.path.append(r'D:\Users\zhangxaochen\Documents\GitHub\curveViewer')
from utils import Keys
from glob import glob

'指定某文件夹, 目录中xml都是用作标定的数据'
calibDatPath = r'D:\Users\zhangxaochen\Documents\axxb-data\cb-imu-handeye-20160120'
calibDatPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8'
newXmlPath = os.path.join(calibDatPath, 'nsImuXml')
if not os.path.exists(newXmlPath):
    os.makedirs(newXmlPath)
    
fnames = glob(calibDatPath+'/*.xml')
psr = et.XMLParser(remove_blank_text = True, remove_comments=True)

for fn in fnames:
    '+++++++++++++++先解析旧格式数据, 可包含多硬件 node'
    tree = et.parse(fn, psr)
    rt = tree.getroot() #CaptureSession
    node = rt[0][0] #->Nodes->Node 只取第一个
    phyId = node.get('phyId')
    
    tss, accs, mags, gyros, quas = [],[],[],[],[]
    for dat in node:
        att = dat.attrib
        dtStr = att['realtime']
        dtObj = datetime.strptime(dtStr, '%Y-%m-%d %H:%M:%S.%f')
        ts = (dtObj-datetime(1970,1,1)).total_seconds() #就按秒，不转到毫秒
        tss.append(ts)
        '【注意】：都是字符串, 不是浮点数'
        accs.append([att['Ax'],att['Ay'],att['Az']])
        mags.append([att['Mx'],att['My'],att['Mz']])
        gyros.append([att['Gx'], att['Gy'], att['Gz']])
        quas.append([att['Rx'],att['Ry'],att['Rz'],att['Rw'],])
        pass
    
    assert len(accs)==len(mags)==len(gyros)==len(quas) #理论上四个 list 应该等长

    '+++++++++++++++后生成新格式数据, 单一硬件 node'
    rt = et.Element(Keys.kSession)
    #---------------root's children
    begtNode = et.SubElement(rt, Keys.kBeginTime)
    begtNode.text = str(tss[0])
    endtNode = et.SubElement(rt, Keys.kEndTime)
    endtNode.text = str(tss[-1])
    threadCountNode = et.SubElement(rt, Keys.kThreadCount)
    threadCountNode.text = str(1)
    threadsNode = et.SubElement(rt, Keys.kThreads)
    #------------------threadsNode' children
    threadNode = et.SubElement(threadsNode, Keys.kThread)
    
    #------------------threadNode' children
    threadNameNode = et.SubElement(threadNode, Keys.kName)
    '这次是真实 phyId了, 以前手机上全都fake "0"'
    #threadNameNode.text = phyId
    '算了，为了与王镇 load_acc_data 保持兼容, 仍然fake=0'
    threadNameNode.text = str(0)
    
    channelCountNode = et.SubElement(threadNode, Keys.kChannelCount)
    channelCountNode.text = '4'
    channelsNode = et.SubElement(threadNode, Keys.kChannels)
    
    #------------------channelsNode' children
    #ckeys=[Keys.kA, Keys.kG, Keys.kM, Keys.kR]
    '全部按照amgr顺序：'
    cnames = list('amgr')
    datLsts = [accs, mags, gyros, quas]
    
    #cnNodes = []
    for cname, dataLst in zip(cnames, datLsts):
        #------------------channelNode' children
        cnNode = et.SubElement(channelsNode, Keys.kChannel)
        cnNameNode = et.SubElement(cnNode, Keys.kName)
        cnNameNode.text = cname
        
        frameCountNode = et.SubElement(cnNode, Keys.kFrameCount)
        frameCountNode.text = str(len(tss))
        
        framesNode = et.SubElement(cnNode, Keys.kFrames)
        
        #------------------framesNode' children
        for i, tup34 in enumerate(dataLst):
            frameNode = et.SubElement(framesNode, Keys.kFrame)
            
            valueNode = et.SubElement(frameNode, Keys.kValue)
            xNode = et.SubElement(valueNode, Keys.kX)
            xNode.text = str(tup34[0])
            yNode = et.SubElement(valueNode, Keys.kY)
            yNode.text = str(tup34[1])
            zNode = et.SubElement(valueNode, Keys.kZ)
            zNode.text = str(tup34[2])
            
            if cname=='r':
                wNode = et.SubElement(valueNode, Keys.kW)
                wNode.text = str(tup34[3])
            
            'amgr 是帧同步的, 都用同一个 index, time'
            indexNode = et.SubElement(frameNode, Keys.kIndex)
            indexNode.text = str(i)
            timeNode = et.SubElement(frameNode, Keys.kTime)
            timeNode.text = str(tss[i])
            pass
        
        
        #cnNodes.append(cnNode)
        pass
    
#     accCnNode = et.SubElement(channelsNode, Keys.kChannel)
#     channelNameNode = et.SubElement(accCnNode)
#     for a, m, g, r in zip(accs, mags, gyros, quas):
        
    xmlStr = et.tostring(rt, pretty_print=True, xml_declaration=True, encoding='utf-8').decode()
    
    fpath, fname = os.path.split(fn)
    basefn, ext = os.path.splitext(fname)
    newXmlFname = os.path.join(newXmlPath, basefn + '.ns' + ext)
    print 'newXmlFname:', newXmlFname
    with open(newXmlFname, 'w') as f:
        f.write(xmlStr)
    pass


newXmlFname: D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\nsImuXml\000.ns.xml
newXmlFname: D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\nsImuXml\001.ns.xml
newXmlFname: D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\nsImuXml\002.ns.xml
newXmlFname: D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\nsImuXml\003.ns.xml
newXmlFname: D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\nsImuXml\004.ns.xml
newXmlFname: D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\nsImuXml\005.ns.xml
newXmlFname: D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\nsImuXml\006.ns.xml
newXmlFname: D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\nsImuXml\007.ns.xml
newXmlFname: D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\nsImuXml\008.ns.xml
newXmlFname: D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\nsImuXml\009.ns.xml
newXmlFname: D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\nsImuXml\010.ns.xml

In [3]:
'''验证任光阔 rgkAccCalib 输出的校正系数的效果 //2016-1-27 13:25:46
'''

from lxml import etree as et
import sys

fn = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\010.xml'
fn = r'D:/Users/zhangxaochen/Documents/axxb-data/cb-imu-handeye-20160120/CaptureSession.cb-imu-handeye-20160120.xml'

psr = et.XMLParser(remove_blank_text = True, remove_comments=True)
tree = et.parse(fn, psr)
rt = tree.getroot() #CaptureSession
node = rt.find('Nodes').find('Node') #->Nodes->Node 只取第一个
phyId = node.get('phyId')

tss, accs, mags, gyros, quas = [],[],[],[],[]
for dat in node:
    att = dat.attrib
    dtStr = att['realtime']
    dtObj = datetime.strptime(dtStr, '%Y-%m-%d %H:%M:%S.%f')
    ts = (dtObj-datetime(1970,1,1)).total_seconds() #就按秒，不转到毫秒
    tss.append(ts) #时间戳字符串

    '用浮点数， 不用字符串：'
    accs.append([float(att['Ax']), float(att['Ay']), float(att['Az'])])
    pass

accs = np.array(accs)

'calibrated accs:'


calibFn = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\static.6+8\capg-imu-rgkAccCalib.txt'
coeffs=np.loadtxt(calibFn)
cAccs = np.zeros_like(accs)
for i in range(3):
    cAccs[:, i] = accs[:, i]*coeffs[i] + coeffs[i+3]
    pass

plot(accs)
plot(cAccs)

print mean(accs, axis=0)
print mean(cAccs, axis=0)

[ 0.12778973 -0.21609788  0.9919373 ]
[ 0.07518325 -0.14307303  0.90354258]


In [18]:
tfn = r'D:\Users\zhangxaochen\Desktop\q&nq.txt'
# tfn = r'D:\Users\zhangxaochen\Desktop\q&nq-2.txt'
qnq = np.loadtxt(tfn, delimiter=',')
# qnq.shape
plot(qnq[:,:4])
figure()
plot(qnq[:,4:])


In [13]:
tfn = r'D:\Users\zhangxaochen\Desktop\q&nq.txt' #duyu-calib之后
qnq = np.loadtxt(tfn, delimiter=',')
plot(qnq[:,4:])

tfn = r'D:\Users\zhangxaochen\Desktop\q&nq-2.txt' #最原始数据直接计算
qnq = np.loadtxt(tfn, delimiter=',')
plot(qnq[:,4:])
legend('abcdxyzw')



In [94]:
i=582
print dQuas[i], newDquasImu[i]
print tf.q2m(dQuas[i]), tf.q2m(newDquasImu[i])

[ 0.33097704  0.15367496 -0.02542681  0.93069419] [ 0.34283096  0.12608279 -0.0028805   0.93089298]
[[ 0.95147497  0.14905493  0.26921741  0.        ]
 [ 0.05439661  0.77961535 -0.62389174  0.        ]
 [-0.30288017  0.60826189  0.73367641  0.        ]
 [ 0.          0.          0.          1.        ]] [[ 0.96818966  0.09181306  0.23276413  0.        ]
 [ 0.08108729  0.76491726 -0.63900425  0.        ]
 [-0.23671423  0.63755152  0.73314011  0.        ]
 [ 0.          0.          0.          1.        ]]


In [44]:
# subplot(211)
plot(dQuas)
# legend(list('xyzw'))

# subplot(212)
# plot(dQuasImu)
# legend(list('xyzw'))
# legend(list('xyzwabcd'))

# idealx=np.array([[0,1,0,0],[0,0,-1,0],[-1,0,0,0],[0,0,0,1]])
idealx=np.array([[0,1,0],[0,0,-1],[-1,0,0]])#,[0,0,0,1]])
idealx=np.array([[-0.2790, 0.8740, 0.3978],
                 [-0.1239, 0.3781, -0.9175],
                 [-0.9523, -0.3052, 0.0028],
])
t=np.eye(4)
t[:3,:3]=idealx
idealx=t
qx = tf.m2q(idealx)

newDquasImu = []
for qimu in dQuasImu:
    newq = tf.qmul(qx, tf.qmul(qimu, tf.qinv(qx)))
    #newq = tf.qmul(qx, tf.qmul(tf.qinv(qimu), tf.qinv(qx)))
    newDquasImu.append(newq)
    pass

newDquasImu = np.array(newDquasImu)

figure()
plot(newDquasImu)


In [19]:
idealx=np.array([[0,1,0,0],[0,0,-1,0],[-1,0,0,0],[0,0,0,1]])
qx = tf.m2q(idealx)
qx

array([ 0.5,  0.5, -0.5,  0.5])

In [ ]:
'最后释放oni资源'
depth_stream.stop()
openni2.unload()